In [1]:
# ! wget -q https://omnomnom.vision.rwth-aachen.de/data/metrabs/metrabs_multiperson_smpl_combined.zip -P models
# ! unzip -q 'models/*.zip' -d models

In [2]:
import tensorflow as tf
import time

In [3]:
import cv2

In [4]:
with tf.device('/device:GPU:0'):
    t1 = time.time()
    model = tf.saved_model.load('models/metrabs_multiperson_smpl_combined')
    t2 = time.time()
    print('Total time to load model :', (t2-t1))
    

Total time to load model : 16.876500129699707


In [5]:
data_path='vera.mp4'

In [6]:
def video_predict(data_path):
    det=[]
    pose_3d=[]
    pose_2d=[]
    frames=[]
    joints=[]
    cap = cv2.VideoCapture(data_path)
    while True:
        flag, frame = cap.read()
        height, width = frame.shape[:2]
        if height < width:
              frame = cv2.rotate(frame, cv2.ROTATE_90_CLOCKWISE) 
        frame = tf.convert_to_tensor(frame)
        with tf.device('/device:GPU:0'):
            detections, poses3d, poses2d = model.predict_single_image(frame)
            det.append(detections)
            pose_3d.append(poses3d)
            pose_2d.append(poses2d)
            frames.append(frame)
            joints.append(model.joint_edges.numpy())
        
        if flag:
        # The frame is ready and already captured
            pos_frame = cap.get(cv2.CAP_PROP_POS_FRAMES)
            #print(str(pos_frame)+" frames")
            cv2.destroyAllWindows()
        else:
        # The next frame is not ready, so we try to read it again
            cap.set(cv2.CAP_PROP_POS_FRAMES, pos_frame-1)
            print( "frame is not ready")
        # It is better to wait for a while for the next frame to be ready
            cv2.waitKey(1000)
        if cv2.waitKey(10) == 27:
            break
        if cap.get(cv2.CAP_PROP_POS_FRAMES) == cap.get(cv2.CAP_PROP_FRAME_COUNT):
        # If the number of captured frames is equal to the total number of frames,
        # we stop
            break
    return frames, det, pose_3d, pose_2d, joints

In [7]:
t1 = time.time()
frames, det, pose_3d, pose_2d, joints = video_predict(data_path)
t2 = time.time()
print('Total time to get predictions :', (t2-t1))

Total time to get predictions : 33.442110538482666


In [8]:
#тут только нужные точки
import numpy as np
poses3d=[]
idx = [14,15,16, 17, 18, 19, 20, 0,1,3,4,6,7,9,10,21,22,23,2]
for pose in pose_3d:
    poses3d.append(pose.numpy())

In [9]:
from enum import IntEnum

In [10]:
class Keypoints(IntEnum):
    Head = 0
    LShoulder = 1
    RShoulder  = 2
    LElbow = 3
    RElbow = 4
    LWrist = 5
    RWrist = 6 
    LHip = 7
    RHip = 8
    LKnee = 9
    RKnee = 10
    LAnkle = 11 
    RAnkle = 12
    LToe = 13
    RToe = 14
    LHan = 15
    RHan = 16
    Pelv = 17
    Bell = 18
#     ClubFace = 19

In [11]:
import os
import csv
import pandas as pd

def writer(poses3d,data_path):
    csv_path='csv_dir'
    file=data_path.split(".")[0]+'.csv'

    try:
        if not os.path.exists(csv_path):
            os.mkdir(csv_path)
    except Exception as e:
        print(repr(e))
        raise e
        
    with open('csv_dir/{}'.format(file), mode='w') as csv_file:
        csv_writer = csv.writer(csv_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        csv_writer.writerow(['frame_no',
                         'Head.x', 'Head.y', 'Head.z'
                         'LShoulder.x', ' LShoulder.y', ' LShoulder.z',
                         'RShoulder.x', 'RShoulder.y', 'RShoulder.z',
                         'LElbow.x', 'LElbow.y', 'LElbow.z',
                         'RElbow.x', 'RElbow.y', 'RElbow.z',
                         'LWrist.x', 'LWrist.y', 'LWrist.z',
                         'RWrist.x', 'RWrist.y', 'RWrist.z',
                         'LHip.x', 'LHip.y', 'LHip.z',
                         'RHip.x', 'RHip.y', 'RHip.z',
                         'LKnee.x', 'LKnee.y', 'LKnee.z',
                         'RKnee.x', 'RKnee.y', 'RKnee.z',
                         'LAnkle.x', 'LAnkle.y', 'LAnkle.z',
                         'RAnkle.x', 'RAnkle.y', 'RAnkle.z',
                         'LToe.x', 'LToe.y', 'LToe.z',
                         'RToe.x', 'RToe.y', 'RToe.z',
                         'LHan.x', 'LHan.y', 'LHan.z',
                         'RHan.x', 'RHan.y', 'RHan.z',
                         'Pelv.x', 'Pelv.y', 'Pelv.z',
                         'Bell.x', 'Bell.y', 'Bell.z'
#                          'spin.x', 'spin.y', 'spin.z',
#                          'thor.x', 'thor.y', 'thor.z',
#                          'neck.x', 'neck.y', 'neck.z',
#                          'lcla.x', 'lcla.y', 'lcla.z',
#                          'rcla.x', 'rcla.y', 'rcla.z',
#                          'ClubFace.x', 'ClubFace.y', 'ClubFace.z'
                         ])
        
        for frame, pose in enumerate(poses3d):
            row = [frame] + ['' for _ in range(57)]  # Не забыть поправить когда добавлю клюшку Number of coco points * 3 -> 19 * 3 -> 57
            p=pose[0]
            for part in Keypoints:
                index = 1 + 3 * part.value  # Index at which the values for this joint would start in the final row
                row[index] = p[part.value][0]
                row[index + 1] = p[part.value][1] 
                row[index + 2] = p[part.value][2]
            csv_writer.writerow(row)

In [12]:
# writer(poses3d,'test')

In [13]:
def load_csv(path):
    """Load keypoint coordinates stored in a CSV file.
    Columns are in order frame_no, nose.(x|y|p), (l|r)eye.(x|y|p), (l|r)ear.(x|y|p), (l|r)shoulder.(x|y|p),
    (l|r)elbow.(x|y|p), (l|r)wrist.(x|y|p), (l|r)hip.(x|y|p), (l|r)knee.(x|y|p), (l|r)ankle.(x|y|p)
    l - Left side of the identified joint
    r - Right side of the identified joint
    x - X coordinate of the identified joint
    y - Y coordinate of the identified joint
    p - Probability of the identified joint
    Coordinate list for a frame = [ [x, y], [x, y], [x, y], ... ]
    *coordinates in order specified in the CSV header
    Returns a list of coordinates for each frame = [ [...], [...], ... ]
    :param csv_fp: Path to the CSV file
    """
    import csv
    
    pose_coordinates = []

    with open(path, 'r') as csv_file:
        reader = csv.reader(csv_file, delimiter=',')
        next(reader)

        for row in reader:
            # TODO: Figure out a way to handle missing joints
            coordinates = []  # List to store XY coordinates of every joint

            # Fill up the coordinate list for a single frame
            for index in range(1, len(row), 3):  # Starting from 1 to skip frame column, 3 because joint.x|y|prob
                coordinates.append([float(row[index]), float(row[index + 1]),float(row[index + 2])])

            # Fill up the final list with coordinate lists of all the frames
            pose_coordinates.append(coordinates)

    return pose_coordinates

In [14]:
# all_coordinates=load_csv('csv_dir/test.csv')

In [15]:
def normalise(all_coordinates):
    """The normalization is a simple coordinate transformation done in two steps:
    1. Translation: All the key points are translated such that the nose key point becomes the origin of the coordinate
        system. This is achieved by subtracting the nose key points coordinates from all other key points.
    2. Scaling: The key points are scaled such that the distance between the left shoulder and right shoulder key point
        becomes 1. This is done by dividing all key points coordinates by the distance between the left and right
        shoulder key point.
    """
    
    import math
    import numpy
    
    from math import sqrt
    
    norm_coords = []  # Hold the normalised coordinates for every frame

    # Iterate over every frame
    for coordinates in all_coordinates:
        # Step 1: Translate
        
        coordinates = [
            [coordinate[0] - coordinates[Keypoints.Head.value][0], coordinate[1] - coordinates[Keypoints.Head.value][1],coordinate[2] - coordinates[Keypoints.Head.value][2]]
            for coordinate in coordinates
        ]
        
        # Step 2: Scale
        x= coordinates[Keypoints.LShoulder.value][0] - coordinates[Keypoints.RShoulder.value][0]
        y= coordinates[Keypoints.LShoulder.value][1] - coordinates[Keypoints.RShoulder.value][1]
        z= coordinates[Keypoints.LShoulder.value][2] - coordinates[Keypoints.RShoulder.value][2]
        dist=sqrt(x*x+y*y+z*z)
#         dist = math.hypot(coordinates[Keypoints.LShoulder.value][0] - coordinates[Keypoints.RShoulder.value][0],
#                           coordinates[Keypoints.LShoulder.value][1] - coordinates[Keypoints.RShoulder.value][1],
#                           coordinates[Keypoints.LShoulder.value][2] - coordinates[Keypoints.RShoulder.value][2])
#         print(dist)
        coordinates = [[coordinate[0] / dist, coordinate[1] / dist, coordinate[2] / dist] for coordinate in coordinates]

        norm_coords.append(coordinates)

    return norm_coords

In [16]:
# n=normalise(all_coordinates)

In [21]:
import csv
import math
import sys
from itertools import chain
# from typing import List

import fastdtw
import numpy as np

from scipy.ndimage.filters import gaussian_filter
from scipy.signal import medfilt
from scipy.spatial.distance import euclidean


def calculate_score(seq1, seq2, dimensions):
    """Calculate how similar the two pose sequences are."""

    def process_signal(signal):
        """Final processing before dynamic time warping."""
        # Apply Gaussian filter for further processing
        signal = gaussian_filter(signal, sigma=1)

        # Make the sequence/signal zero-mean by subtracting the mean from it
        mean = np.mean(signal)
        return [x - mean for x in signal]

    distance = 0.0

    for dim in dimensions:
        sig1 = process_signal(signal=seq1[dim])
        sig2 = process_signal(signal=seq2[dim])

        temp_distance, _ = fastdtw.fastdtw(sig1, sig2, radius=30, dist=euclidean)
        distance += temp_distance

    # Normalise DTW score
    distance /= len(dimensions)

    return distance



In [22]:
def dimension_selection(frames):
    """Remove indices that don't vary a lot during the pose.
    Key points that do not move significantly in the sequence will cause the signals of the respective coordinates to be
    roughly constant with only little variance. All signals whose variance is below a threshold will be filtered out and
    are assumed to be uninformative.
    :returns: A set of indices of dimensions that should be kept
    """

#     def keep_sequence(seq: List) -> bool:
#         """Check whether the points in the sequence vary a lot or not."""
#         # Use a median filter for smoothing
#         seq = medfilt(seq, kernel_size=3)

#         # Filter the dimension based on variance
#         return np.var(seq) > 0.10

    # Reorder the coordinates such that they are per joint and not per frame
    frames = [list(chain(*frame)) for frame in frames]  # Flatten the nested lists inside
    sequences = list(map(list, zip(*frames)))  # Transpose the list

    # Drop low variance columns
    dimensions = [i for i, sequence in enumerate(sequences)]

    return dimensions

In [23]:
def main(path_1,path_2):    
    
    pose_csv1 = path_1
    pose_csv2 = path_2

    # Load the CSVs
    pose1 = load_csv(path=pose_csv1)
    pose2 = load_csv(path=pose_csv2)

    # Normalization
    pose1 = normalise(pose1)
    pose2 = normalise(pose2)

    # # Dimension Selection
    pose1_dimensions = dimension_selection(pose1.copy())
    pose2_dimensions = dimension_selection(pose2.copy())
#     print(pose1 + pose2)
    dimensions = sorted(set(pose1_dimensions + pose2_dimensions))  # Take a union to get final list of dimensions

    score = calculate_score(pose1, pose2, dimensions)
    print(f'Score = {score:.6f}')

In [24]:
score=main('csv_dir/test.csv','csv_dir/test.csv')

Score = 0.000000
